In [1]:
from sqlalchemy import create_engine
import pandas as pd 
import json

with open('/home/douglas/postgres_credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
    address = data['address']

conn = create_engine('postgresql://{}:{}@{}:5432/ex_employee1'.format(username, password, address))

/home/douglas/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
pd.read_sql_query("SELECT * FROM employee LIMIT 5", conn)

,empid,name,managerid,deptid,salary
0,1,Micheal Scott,NaN,1,6000.0
1,2,Dwight Schrute,1.0,2,4000.0
2,3,Jim Halpert,1.0,2,3500.0
3,4,Anthony Stark,NaN,1,100000.0
4,5,Bill Nye,4.0,5,3800.0


In [10]:
pd.read_sql_query("SELECT * FROM department", conn)

,deptid,deptname
0,1,Admin
1,2,Sales
2,3,IT
3,4,Training
4,5,Research
5,6,Legal
6,7,SecretOps


In [5]:
for r in conn.execute("SELECT COUNT(*) FROM employee", conn):
    print(r)
for r in conn.execute("SELECT COUNT(*) FROM department", conn):
    print(r)

(16,)
(7,)


# Problems

In [6]:
# List all the employees in order of descreasing salary
pd.read_sql_query("SELECT * FROM employee ORDER BY salary desc", conn)

,empid,name,managerid,deptid,salary
0,4,Anthony Stark,NaN,1,100000.0
1,15,Joe McMillan,13.0,2,80000.0
2,8,Shuri,NaN,5,80000.0
3,14,Cameron Howe,13.0,3,75000.0
4,13,Donna Clarke,NaN,1,75000.0
5,9,Diane Rawlinson,NaN,1,50000.0
6,11,Nancy Karrigan,9.0,4,42000.0
7,16,Saul Goodman,13.0,6,40000.0
8,12,Kristi Yamaguchi,9.0,4,40000.0
9,10,Tonya Harding,9.0,4,30000.0


In [11]:
# List all the department names, and the number of employees in that department. Order by number of employess in department (greatest to least)
pd.read_sql_query('''SELECT d.deptname, count(e.empid) AS num_in_dept
                  FROM department d 
                  LEFT JOIN employee e
                  ON d.deptid = e.deptid
                  GROUP BY 1
                  ORDER BY 2 desc''', conn)
# count(*) vs count(col)
# get all rows vs get all non nulls

,deptname,num_in_dept
0,Research,4
1,Admin,4
2,Training,3
3,Sales,3
4,Legal,1
5,IT,1
6,SecretOps,0


In [13]:
# List all the employees that don't have a manager
pd.read_sql_query("SELECT * FROM employee WHERE managerid IS NULL", conn)

,empid,name,managerid,deptid,salary
0,1,Micheal Scott,None,1,6000.0
1,4,Anthony Stark,None,1,100000.0
2,8,Shuri,None,5,80000.0
3,9,Diane Rawlinson,None,1,50000.0
4,13,Donna Clarke,None,1,75000.0


In [15]:
# List all employees by name, and the name of their manager. If the employee doesn't have a manager, leave the column as NULL.
pd.read_sql_query('''SELECT e1.name AS employee, e2.name AS manager
                  FROM  employee e1
                  LEFT JOIN employee e2 
                  ON e1.managerid = e2.empid
                  ''', conn)
# self join

,employee,manager
0,Dwight Schrute,Micheal Scott
1,Jim Halpert,Micheal Scott
2,Bruce Banner,Anthony Stark
3,Bill Nye,Anthony Stark
4,Jane Foster,Anthony Stark
5,Nancy Karrigan,Diane Rawlinson
6,Tonya Harding,Diane Rawlinson
7,Kristi Yamaguchi,Diane Rawlinson
8,Cameron Howe,Donna Clarke
9,Saul Goodman,Donna Clarke


In [19]:
# For each manager, list the number of employees he or she is managing. For these purposes, a manager is anyone who is not managed by someone else, even if that person has no direct reports.
pd.read_sql_query('''SELECT e1.name AS name, count(e2.empid) num_managed
                  FROM  employee e1
                  LEFT JOIN employee e2 
                  ON e1.empid = e2.managerid
                  WHERE e1.managerid IS NULL
                  GROUP BY 1
                  ORDER BY 2 desc
                  ''', conn)

,name,num_managed
0,Anthony Stark,3
1,Diane Rawlinson,3
2,Donna Clarke,3
3,Micheal Scott,2
4,Shuri,0


In [23]:
# Find the two highest paid people per department
pd.read_sql_query('''
SELECT name, deptid, salary, rank 
FROM (SELECT name AS name, deptid, salary,
                    Rank() OVER (PARTITION BY deptid ORDER BY salary desc)
                  FROM  employee ) s
                  WHERE rank IN (1,2)
                
                  ''', conn)

,name,deptid,salary,rank
0,Anthony Stark,1,100000.0,1
1,Donna Clarke,1,75000.0,2
2,Joe McMillan,2,80000.0,1
3,Dwight Schrute,2,4000.0,2
4,Cameron Howe,3,75000.0,1
5,Nancy Karrigan,4,42000.0,1
6,Kristi Yamaguchi,4,40000.0,2
7,Shuri,5,80000.0,1
8,Bruce Banner,5,8000.0,2
9,Saul Goodman,6,40000.0,1
